Actividad 1 

Explicacion del preprocesamiento hecho y/o fotos del preprocesamiento del csv

Actividad 2 

En este caso se usó <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html">KFold</a>. Recuerde que también existe <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html">train_test_split</a>.


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

# Cargar los datos desde los archivos CSV
archivo_taller = 'Taller_2_Titulacion_DatosTaller.csv'
archivo_eval = 'Taller_2_Titulacion_Evaluación.csv'
datos_taller = pd.read_csv(archivo_taller, encoding='latin-1', delimiter=';')
datos_evaluacion = pd.read_csv(archivo_eval, encoding='latin-1', delimiter=';')

# Separar las características y la variable objetivo
X_taller = datos_taller.iloc[:, :-1]  
y_taller = datos_taller.iloc[:, -1]

# Eliminar la columna "Id" de ambos conjuntos de datos
X_taller = X_taller.drop('Id', axis=1)
X_evaluacion = datos_evaluacion.drop('Id', axis=1)

# Convertir variables categóricas en dummies
X_taller = pd.get_dummies(X_taller)
X_evaluacion = pd.get_dummies(X_evaluacion)

# Asegurar que las columnas coincidan en ambos conjuntos de datos
X_taller, X_evaluacion = X_taller.align(X_evaluacion, join='left', axis=1, fill_value=0)

# Convertir las etiquetas categóricas en valores binarios
y_taller = y_taller.map({"chileno": 1, "extranjero": 0, "mujer": 1, "hombre": 0}).fillna(0)

# Reemplazar NaN por 0 en el conjunto de entrenamiento
X_taller = X_taller.fillna(0)
X_evaluacion = X_evaluacion.fillna(0)

# Crear objeto KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Visualizar las primeras filas de los datos procesados
print(X_taller.head())
print(y_taller.head())


   MAT_1SEM_PROM  FIS_1SEM_PROM  ING_1SEM_PROM  ACTF_1SEM_A  ACTF_1SEM_R  \
0           55.0            0.0           79.0          1.0          0.0   
1           56.0           57.0           88.0          0.0          0.0   
2           55.0           64.0           55.0          0.0          0.0   
3            0.0           56.0            0.0          0.0          0.0   
4           87.0           82.0           86.0          0.0          0.0   

   OTRANS_1SEM_A  OTRANS_1SEM_R  OTRANS_1SEM_PROM  ESP_1SEM_A  ESP_1SEM_R  \
0            2.0            0.0              67.0         2.0         0.0   
1            0.0            0.0               0.0         4.0         0.0   
2            0.0            0.0               0.0         4.0         0.0   
3            2.0            0.0              64.0         2.0         0.0   
4            0.0            0.0               0.0         3.0         1.0   

   ...  descripcion_rama_actividad_madre_Explotación de minas y canteras  \
0  .

Actividad 3

En este caso debieran que ser 4 bloques 2 modelos y dos modificaciones de parametros para cada modelo 

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler 

# Cargar los datos desde el archivo CSV
archivo_taller = 'Taller_2_Titulacion_DatosTaller.csv'
datos_taller = pd.read_csv(archivo_taller, encoding='latin-1', delimiter=';')

# Separar las características y la variable objetivo
X = datos_taller.iloc[:, :-1]  
y = datos_taller.iloc[:, -1]

# Eliminar la columna "Id"
X = X.drop('Id', axis=1)

# Convertir variables categóricas en dummies si es necesario
X = pd.get_dummies(X)

# Convertir las etiquetas categóricas en valores binarios
y_taller = y_taller.map({"chileno": 1, "extranjero": 0, "mujer": 1, "hombre": 0}).fillna(0)

# Reemplazar NaN por 0 en el conjunto de entrenamiento
X_taller = X_taller.fillna(0)
X_evaluacion = X_evaluacion.fillna(0)
#------------------------------------------parte de ACT2

# Imputar valores faltantes con la media de la columna
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Escalar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Imputar valores faltantes en la variable objetivo
y_imputer = SimpleImputer(strategy='most_frequent')
y_imputed = y_imputer.fit_transform(y.values.reshape(-1, 1)).flatten()

# Crear objeto KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Definir los modelos y configuraciones de hiperparámetros
models = [
    ('Decision Tree', DecisionTreeClassifier()),
    ('Random Forest', RandomForestClassifier()),
    ('Logistic Regression', LogisticRegression(max_iter=5000))
]

# Iterar sobre los modelos
for name, model in models:
    print(f'Modelo: {name}')
    accuracies = []
    for train_index, test_index in kf.split(X_imputed):
        X_train, X_test = X_imputed[train_index], X_imputed[test_index]
        y_train, y_test = y_imputed[train_index], y_imputed[test_index]

        # Entrenar el modelo
        model.fit(X_train, y_train)

        # Evaluar el modelo
        accuracy = cross_val_score(model, X_test, y_test, cv=5, scoring='accuracy')
        accuracies.append(accuracy.mean())

    print(f'Precisión promedio: {np.mean(accuracies):.4f}\n')



NameError: name 'X' is not defined

Actividad 4

Aplicacion del modelo a nuevo conjunto

In [ ]:
import pandas as pd

# Cargar los datos de evaluación
datos_evaluacion = pd.read_csv('Taller_2_Titulacion_Evaluación.csv', encoding='latin-1', delimiter=';')

# Eliminar la columna "Id" si es necesario
X_evaluacion = datos_evaluacion.drop('Id', axis=1)

# Convertir variables categóricas en dummies si es necesario
X_evaluacion = pd.get_dummies(X_evaluacion)

# Hacer predicciones con el modelo cargado
predictions = model.predict(X_evaluacion)

# Convertir las predicciones a etiquetas legibles
predicted_labels = np.where(predictions[:, 1], 'SÍ', 'NO')  # Si la segunda columna (índice 1) es True, entonces 'SÍ', de lo contrario 'NO'

# Crear un DataFrame con las columnas Id y Prediccion
resultados = pd.DataFrame({'Id': datos_evaluacion['Id'], 'Prediccion': predicted_labels})

# Guardar el DataFrame como un archivo de texto separado por comas
resultados.to_csv('clasificacion_titulo.txt', index=False, sep=',', header=True)


c:\Users\Snail\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values